##  Installing and Importing Necessary Libraries

In [1]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.73 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 88.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 283.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 265.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 423.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 341.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 236.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_ve

**Note**:
- After running the above cell, kindly restart the runtime and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [2]:
!pip install langchain==0.3.27 \
            langchain-community==0.3.27 \
            torch==2.8.0+cu126 \
            sentence_transformers==5.1.1 \
            faiss-cpu==1.12.0 \
            pypdf==6.1.3 \
            huggingface_hub==0.35.3

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached faiss_cpu-1.12.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached pypdf-6.1.3-py3-none-any.whl.metadata (7.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00


**Note**:
- After running the above cell, kindly restart the runtime and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [1]:
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp
# from langchain.llms import Ollama

## Load the data

In [2]:
#  ADD PATH
loader = TextLoader('AAPL-MDA.txt')
data = loader.load()

## Split the Extracted Data into Text Chunks

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

text_chunks = text_splitter.split_documents(data)

In [4]:
len(text_chunks)

715

In [5]:
#get the third chunk
text_chunks[3].page_content

'and Apple Watch. Services Services net sales increased during 2020 compared to 2019 due primarily to higher net sales from the App Store, advertising and cloud services. Apple Inc. | 2020 Form 10-K | 21 Segment Operating Performance The Company manages its business primarily on a geographic basis. The Company’s reportable segments consist of the Americas, Europe, Greater China, Japan and Rest of Asia Pacific. Americas includes both North and South America. Europe includes European countries, as well as India, the Middle East and Africa. Greater China includes China mainland, Hong Kong and Taiwan. Rest of Asia Pacific includes Australia and those Asian countries not included in the Company’s other reportable segments. Although the reportable segments provide similar hardware and software products and similar services, each one is managed separately to better align with the location of the Company’s customers and distribution partners and the unique market dynamics of each geographic re

In [6]:
text_chunks[2].page_content

'paid dividends and dividend equivalents of $14.1 billion. On August 28, 2020, the Company effected a four-for-one stock split to shareholders of record as of August 24, 2020. All share, RSU and per share or per RSU information has been retroactively adjusted to reflect the stock split. Apple Inc. | 2020 Form 10-K | 20 Products and Services Performance The following table shows net sales by category for 2020, 2019 and 2018 (dollars in millions): (1)Products net sales include amortization of the deferred value of unspecified software upgrade rights, which are bundled in the sales price of the respective product. (2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch and Apple-branded and third-party accessories. (3)Services net sales include sales from the Company’s advertising, AppleCare, digital content and other services. Services net sales also include amortization of the deferred value of Maps, Siri, and fre

In [7]:
len(text_chunks[22].page_content)

1992

## Load the embedding model

In [8]:
# https://huggingface.co/spaces/mteb/leaderboard
embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/tmp/ipython-input-752314550.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Create Embeddings for each of the Text Chunk

In [9]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [10]:
text_chunks[0].page_content

'Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations The following discussion should be read in conjunction with the consolidated financial statements and accompanying notes included in Part II, Item 8 of this Form 10-K. This section of this Form 10-K generally discusses 2020 and 2019 items and year-to-year comparisons between 2020 and 2019. Discussions of 2018 items and year-to-year comparisons between 2019 and 2018 that are not included in this Form 10-K can be found in “Management’s Discussion and Analysis of Financial Condition and Results of Operations” in Part II, Item 7 of the Company’s Annual Report on Form 10-K for the fiscal year ended September 28, 2019. Fiscal Year Highlights COVID-19 Update COVID-19 has spread rapidly throughout the world, prompting governments and businesses to take unprecedented measures in response. Such measures have included restrictions on travel and business operations, temporary closures of businesses, and 

In [11]:
vector_store.index.reconstruct(0)

array([ 4.16787341e-03, -1.48787340e-02, -8.85816757e-03, -1.56945474e-02,
        7.74797499e-02, -3.21615976e-03,  2.07387730e-02,  6.89300522e-02,
       -1.58986300e-02, -1.77491140e-02, -4.94190007e-02, -2.90449671e-02,
       -6.10513352e-02,  4.57197540e-02, -1.75454412e-02,  3.32643427e-02,
        5.34815527e-02,  2.74204910e-02,  2.84301862e-02,  1.22596830e-04,
       -6.59145042e-02, -4.30929996e-02,  5.00659049e-02,  6.72984915e-03,
        5.43527752e-02, -3.26178372e-02,  1.49575407e-02, -1.83257945e-02,
       -4.23273928e-02,  6.82438537e-03,  1.63538719e-03,  2.54531018e-02,
       -9.14115750e-04,  2.26244703e-02,  4.45513763e-02,  8.48012883e-03,
        1.79633200e-02,  1.88731626e-02, -2.90709902e-02, -3.47838737e-02,
       -4.17627990e-02,  1.83200161e-03, -5.58526441e-02, -1.81655604e-02,
       -1.58502646e-02,  2.15283339e-03, -7.14306859e-03,  1.73524953e-02,
       -1.08003784e-02, -3.33475955e-02, -3.47001366e-02, -1.30684525e-02,
       -2.43773367e-02, -

## Load the LLM

In [12]:
# Define the model repository and the file name from HuggingFace Hub
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"

# Download the model file from HuggingFace Hub
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [13]:
# Initialize the LlamaCpp model with configuration
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.01,
    top_p=0.95,
    verbose=False,
    n_ctx=4096,
    n_batch=1024,
    n_gpu_layers=43,
)

In [14]:
# https://ollama.com/library
# llm = Ollama(model='llama3.1:70b')

## Build the chain

In [15]:
agent_colab = RetrievalQA.from_chain_type(llm=llm,verbose=True,chain_type="stuff",
                                          retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                          chain_type_kwargs={"verbose": True})

## Run the query

In [16]:
# The Company performs a detailed review of inventory each fiscal quarter that considers multiple factors including demand forecasts, product life cycle status, product development plans, current sales levels, and component cost trends.

query ="How often does the company review inventory, and what is considered in this inventory calculation?"

In [17]:
output_colab = agent_colab.run(query)

/tmp/ipython-input-901690228.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_colab = agent_colab.run(query)




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

have to adjust its allowance for doubtful accounts, which would affect earnings in the period the adjustments were made. Inventory Valuation and Inventory Purchase Commitments The Company must order components for its products and build inventory in advance of product shipments. The Company records a write-down for inventories of components and products, including third-party products held for resale, which have become obsolete or are in excess of anticipated demand or net realizable value. The Company performs a detailed review of inventory each fiscal quarter that considers multiple factors including demand forecasts, product life cycle status, product development plans, current sa

In [18]:
print(output_colab)

 The company reviews inventory quarterly. It considers multiple factors such as demand forecasts, product life cycle status, product development plans, current sales levels, and component cost trends.


# Evaluate the response

In [19]:
# Prompt to evaluate how relevant the answer is to the original question
relevance_rater_system_message = """
You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

# Prompt to evaluate how well the answer is grounded in the provided context
groundedness_rater_system_message = """
You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

In [20]:
# Template for providing evaluation input
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [21]:
def evaluate_groundedness_and_relevance(user_query, generated_answer, llm, retriever):
    """
    Given a query, answer, LLM, and retriever, this function:
    - Retrieves the context chunks used to generate the answer
    - Evaluates groundedness and relevance of the answer
    - Returns both scores and explanations
    """

    # Retrieve relevant context for the question
    docs = retriever.get_relevant_documents(user_query)
    context = " ".join([doc.page_content for doc in docs])

    # Format the input for both evaluation prompts
    filled_prompt = user_message_template.format(
        question=user_query,
        context=context,
        answer=generated_answer
    )

    # Groundedness evaluation prompt
    groundedness_prompt = f"[INST]{groundedness_rater_system_message}\nuser: {filled_prompt}[/INST]"

    # Relevance evaluation prompt
    relevance_prompt = f"[INST]{relevance_rater_system_message}\nuser: {filled_prompt}[/INST]"

    # Call LLM to get groundedness score
    groundedness_response = llm(prompt=groundedness_prompt)

    # Call LLM to get relevance score
    relevance_response = llm(prompt=relevance_prompt)

    return groundedness_response, relevance_response

In [22]:
groundedness, relevance = evaluate_groundedness_and_relevance(query, output_colab, llm, vector_store.as_retriever(search_kwargs={"k": 2}))

/tmp/ipython-input-2665291083.py:10: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_query)
/tmp/ipython-input-2665291083.py:27: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  groundedness_response = llm(prompt=groundedness_prompt)


In [23]:
print(groundedness)
print('\n\n******************************\n\n')
print(relevance)

  Based on the provided information, I would rate the answer as a 4 out of 5 in terms of how well it is derived from the context. The answer provides a brief summary of the company's inventory review process and the factors considered during each quarterly review. This is consistent with the information provided in the context, which highlights the company's detailed review of inventory each fiscal quarter that considers multiple factors.

However, the answer does not fully address the specific question asked, which was about the frequency of the inventory reviews. The context provides this information by stating that the company performs a detailed review of inventory each fiscal quarter, but the answer does not explicitly mention the frequency of these reviews. Therefore, I would deduct one point from the overall rating.

Overall, the answer is well-derived from the context, but could benefit from more explicit mention of the frequency of the inventory reviews.


********************